# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [43]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [44]:
# File to Load 
csv_file = ("../WeatherPy/Output_data/Cities.csv")

# Read file and store into Pandas data frame
city_maps_df = pd.read_csv(csv_file)

city_maps_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Penzance,50.1186,-5.5371,50.00,57,20,16.11,GB,1619436508
1,Grand-Santi,4.2500,-54.3833,73.80,98,100,0.43,GF,1619436508
2,Yellowknife,62.4560,-114.3525,-2.20,71,20,11.50,CA,1619436226
3,Chenghai,23.4632,116.7680,73.40,64,75,11.18,CN,1619436508
4,Arrondissement du Havre,49.5833,0.3333,54.00,58,0,14.97,FR,1619436508
...,...,...,...,...,...,...,...,...,...
550,Boone,39.0334,-92.3335,55.00,81,0,1.01,US,1619436587
551,Khani,41.9563,42.9566,57.20,77,90,12.66,GE,1619436587
552,Mercedes,-34.6515,-59.4307,54.00,99,93,4.03,AR,1619436587
553,Gannan,47.9147,123.5031,64.47,22,100,16.58,CN,1619436587


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [45]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [46]:
# Create variables for map 
locations = city_maps_df[["Lat", "Lng"]].astype(float)
weights = city_maps_df["Humidity"]

In [47]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [54]:
reduced_weather_df = city_maps_df[city_maps_df["Max Temp"]>70]
reduced_weather_df = reduced_weather_df[reduced_weather_df["Max Temp"]<80]
reduced_weather_df = reduced_weather_df[reduced_weather_df["Wind Speed"]<10]
reduced_weather_df = reduced_weather_df[reduced_weather_df["Cloudiness"]<10]

reduced_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,Rikitea,-23.1203,-134.9692,76.93,76,9,7.43,PF,1619436508
79,Hermanus,-34.4187,19.2345,73.00,59,0,5.01,ZA,1619436519
81,Port Elizabeth,-33.9180,25.5701,75.20,57,0,8.05,ZA,1619436313
179,Jalu,29.0331,21.5482,77.61,27,0,5.95,LY,1619436534
181,Walvis Bay,-22.9575,14.5053,71.60,60,0,4.61,NaN,1619436535
213,Progreso,21.2833,-89.6667,73.40,83,1,6.91,MX,1619436540
300,Ginda,30.6945,78.4932,78.84,14,1,6.67,IN,1619436551
344,Nago,26.5881,127.9761,71.60,68,1,5.75,JP,1619436559
376,Koppies,-27.2418,27.5742,77.27,12,5,5.50,ZA,1619436564


In [55]:
reduced_weather_df = reduced_weather_df.dropna(how="any")

reduced_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,Rikitea,-23.1203,-134.9692,76.93,76,9,7.43,PF,1619436508
79,Hermanus,-34.4187,19.2345,73.00,59,0,5.01,ZA,1619436519
81,Port Elizabeth,-33.9180,25.5701,75.20,57,0,8.05,ZA,1619436313
179,Jalu,29.0331,21.5482,77.61,27,0,5.95,LY,1619436534
213,Progreso,21.2833,-89.6667,73.40,83,1,6.91,MX,1619436540
300,Ginda,30.6945,78.4932,78.84,14,1,6.67,IN,1619436551
344,Nago,26.5881,127.9761,71.60,68,1,5.75,JP,1619436559
376,Koppies,-27.2418,27.5742,77.27,12,5,5.50,ZA,1619436564


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [66]:
hotel_df = reduced_weather_df
hotel_df['Hotel Name'] = ""
hotel_df = hotel_df.reset_index(drop=True)

hotel_df

<ipython-input-66-dc794af2979f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Rikitea,-23.1203,-134.9692,76.93,76,9,7.43,PF,1619436508,
1,Hermanus,-34.4187,19.2345,73.00,59,0,5.01,ZA,1619436519,
2,Port Elizabeth,-33.9180,25.5701,75.20,57,0,8.05,ZA,1619436313,
3,Jalu,29.0331,21.5482,77.61,27,0,5.95,LY,1619436534,
4,Progreso,21.2833,-89.6667,73.40,83,1,6.91,MX,1619436540,
5,Ginda,30.6945,78.4932,78.84,14,1,6.67,IN,1619436551,
6,Nago,26.5881,127.9761,71.60,68,1,5.75,JP,1619436559,
7,Koppies,-27.2418,27.5742,77.27,12,5,5.50,ZA,1619436564,


In [84]:
# for index, row in hotel_df.iterrows():
#     print(f"index: {index}")
#     print(f"row: {row}")
#     print("-----------")

In [79]:
import json

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {  
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
        
    params["location"]="{},{}".format(hotel_df.loc[index, 'Lat'],hotel_df.loc[index, 'Lng'])
    
    response = requests.get(base_url, params=params).json()
    
    try:
    
        hotel_name = response["results"][0]["name"]
    
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
        
    except:
        print("Missing field...skipping")
        
hotel_df.head()
    

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Rikitea,-23.1203,-134.9692,76.93,76,9,7.43,PF,1619436508,Pension Maro'i
1,Hermanus,-34.4187,19.2345,73.00,59,0,5.01,ZA,1619436519,Aloe House Guest Lodge
2,Port Elizabeth,-33.9180,25.5701,75.20,57,0,8.05,ZA,1619436313,Ameera's Villa
3,Jalu,29.0331,21.5482,77.61,27,0,5.95,LY,1619436534,صاحبة الفخامة
4,Progreso,21.2833,-89.6667,73.40,83,1,6.91,MX,1619436540,Hotel Suites Los Cabos


In [87]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]



In [92]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))